In [2]:
import pandas as pd
import numpy as np
import nba_api.stats.endpoints as nba
import nba_api.stats.static.players as players
import nba_api.stats.static.teams as teams
import networkx as nx

In [3]:
df = pd.read_csv('data/active_player_info_team.csv')
df

,index,id,Full_name,Firstname,Surname,Active,team_name,team_id
0,0,1630173,Precious Achiuwa,Precious,Achiuwa,True,Knicks,1610612752
1,1,203500,Steven Adams,Steven,Adams,True,Rockets,1610612745
2,2,1628389,Bam Adebayo,Bam,Adebayo,True,Heat,1610612748
3,3,1630534,Ochai Agbaji,Ochai,Agbaji,True,Raptors,1610612761
4,4,1631231,James Akinjo,James,Akinjo,True,NaN,0
...,...,...,...,...,...,...,...,...
610,610,203469,Cody Zeller,Cody,Zeller,True,Hawks,1610612737
611,611,1627826,Ivica Zubac,Ivica,Zubac,True,Clippers,1610612746
612,612,1641783,Tristan da Silva,Tristan,da Silva,True,Magic,1610612753
613,613,1628427,Vlatko Čančar,Vlatko,Čančar,True,Nuggets,1610612743


In [4]:
players_without_team = df[df['team_id'] == 0]
players_without_team.shape

(47, 8)

In [7]:
team_ids = df['team_id'].unique()
team_network = pd.DataFrame()
for team_id in team_ids:
    players_on_team = df[df['team_id'] == team_id]['id']
    for player_id in players_on_team:
        player_network = pd.DataFrame(columns=['passer_id', 'receiver_id', 'freq', 'team_id'])
        passes_made = nba.playerdashptpass.PlayerDashPtPass(team_id=team_id, player_id=player_id).get_data_frames()[0]
        player_network['receiver_id'] = passes_made['PASS_TEAMMATE_PLAYER_ID']
        player_network['freq'] = passes_made['FREQUENCY']
        player_network['passer_id'] = player_id
        player_network['team_id'] = team_id
        team_network = pd.concat([team_network, player_network], ignore_index=True)


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [7]:
team_network

,passer_id,receiver_id,freq,team_id
0,1628384,1642278,0.007,1610612752
1,1628384,1630579,0.032,1610612752
2,1628384,1630574,0.005,1610612752
3,1628384,1630540,0.067,1610612752
4,1628384,1630346,0.002,1610612752
...,...,...,...,...
123,1626157,1628973,0.388,1610612752
124,1626157,1628969,0.113,1610612752
125,1626157,1628404,0.192,1610612752
126,1626157,1628384,0.107,1610612752


In [ ]:
G = nx.DiGraph()
for index, row in team_network.iterrows():
    G.add_node(row['passer_id'], team_id=row['team_id'])
    G.add_node(row['receiver_id'], team_id=row['team_id'])
    G.add_edge(row['passer_id'], row['receiver_id'], weight=row['freq'])

nx.write_gexf(G, 'networks/team_network.gexf')



In [28]:
nba.playerdashptpass.PlayerDashPtPass(team_id=team_id, player_id=player_id).get_data_frames()[0]

,PLAYER_ID,PLAYER_NAME_LAST_FIRST,TEAM_NAME,TEAM_ID,TEAM_ABBREVIATION,PASS_TYPE,G,PASS_TO,PASS_TEAMMATE_PLAYER_ID,FREQUENCY,...,AST,FGM,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3A,FG3_PCT
0,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"Dadiet, Pacôme",1642359,0.004,...,0,0,1,0.000,0,1,0.000,0,0,0.000
1,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"Kolek, Tyler",1642278,0.007,...,0,0,0,0.000,0,0,0.000,0,0,0.000
2,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"McBride, Miles",1630540,0.100,...,1,3,11,0.273,2,4,0.500,1,7,0.143
3,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"Ryan, Matt",1630346,0.004,...,0,0,1,0.000,0,1,0.000,0,0,0.000
4,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"Brunson, Jalen",1628973,0.384,...,8,14,35,0.400,13,29,0.448,1,6,0.167
5,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"Bridges, Mikal",1628969,0.114,...,10,11,16,0.688,6,7,0.857,5,9,0.556
6,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"Hart, Josh",1628404,0.199,...,3,4,14,0.286,3,9,0.333,1,5,0.200
7,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"Anunoby, OG",1628384,0.105,...,12,15,28,0.536,14,21,0.667,1,7,0.143
8,1626157,"Towns, Karl-Anthony",New York Knicks,1610612752,NYK,made,12,"Payne, Cameron",1626166,0.083,...,2,2,8,0.250,0,3,0.000,2,5,0.400


In [5]:
# Show 10 digits of the id

print(ids_teams.where(teams_df['nickname'] == 'Lakers').dropna())

10    1.610613e+09
Name: id, dtype: float64


In [8]:
lastNGames = 10
LeagueID = '00'
Month = 1
OpponentTeamID = 0
PerMode = 'Totals'
PlayerID = '2544' #LeBron James
Season = '2019-20'
SeasonType = 'Regular Season'
TeamID = '1610612747'


lebron_made = nba.playerdashptpass.PlayerDashPtPass(team_id=TeamID, player_id=PlayerID).get_data_frames()[0]
lebron_received = nba.playerdashptpass.PlayerDashPtPass(team_id=TeamID, player_id=PlayerID).get_data_frames()[1]

In [11]:
lebron_made['FREQUENCY'].sum()

1.0